In [1]:
from qibo.quantum_info.quantum_network import QuantumNetwork

from qibo.gates import DepolarizingChannel
from qibo.backends import NumpyBackend
from qibo.quantum_info import random_density_matrix, random_unitary

import numpy as np

backend = NumpyBackend()

## Quantum Network

The `QuantumNetwork` class store the Choi operator as a tensor.
A `QuantumNetwork` object can be

- **A quantum state**

In [2]:
state = random_density_matrix(2)
state_choi = QuantumNetwork(state, (1,2))
print(f'A quantum state is a quantum netowrk of the form {state_choi}')

[Qibo 0.2.3|INFO|2024-01-11 22:40:24]: Using numpy backend on /CPU:0


A quantum state is a quantum netowrk of the form J[1 -> 2]


Note that, the input dimension of a quantum state is 1, which is a trivial space.

- **A quantum channel**

In [3]:
test_ch = DepolarizingChannel(0,0.5)
N = len(test_ch.target_qubits)
partition = (2**N, 2**N)
depolar_choi = QuantumNetwork(test_ch.to_choi(), partition)
print(f'A quantum channel is a quantum netowrk of the form {depolar_choi}')

A quantum channel is a quantum netowrk of the form J[2 -> 2]


- **An observable**

In [4]:
Ob = random_density_matrix(2)*2 -0.5
# `sys_out` is used to specify the system is an output of the network
ob_choi = QuantumNetwork(Ob, (2,1))
print(f'An observable is a quantum netowrk of the form {ob_choi}')

An observable is a quantum netowrk of the form J[2 -> 1]


- **A higher-order quantum operator**

In [5]:
comb = random_density_matrix(16)
comb_choi = QuantumNetwork(comb, (2,2,2,2),sys_out=(False,True,False,True))
print(f'A quantum comb is a quantum netowrk of the form {comb_choi}')

A quantum comb is a quantum netowrk of the form J[2, 2 -> 2, 2]


### Properties

The `QuantumNetwork` class may check the following basic properties:
- `partition`: the partition of the Choi operators. For a Choi operator with partition $(n,m)$, the tensor (`np.ndarray`) is of the shape (n,m,n,m).
- `dim`: the dimension of the Choi operator. For a Choi operator with partition $(n,m)$, the dimension is $nm$.
- `sys_out`: a mask that indicates which system is the output system. This property is not used in the current version of the package. But it will be important for checking if the link product of two Choi operators is valid.

It also has the following properties:
- `is_hermitian`: check if the Choi operator is Hermitian. During the creation of the object, we always check if the Choi operator is Hermitian. If not, we will raise an error.
- `is_psd`: check if the Choi operator is positive semidefinite.

If the network is a quantum channel, it also has the following properties:
- `is_causal`: check if the Choi operator is in sequential causal order. In the channel case, this is equivalent to check if it is trace-preserving. [In general](https://scholar.google.com/scholar_url?url=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.80.022339&hl=en&sa=T&oi=gsb&ct=res&cd=0&d=17758644328595036857&ei=DG1sZaWhAbqR6rQPscmLsAU&scisig=AFWwaeZZhwUe_FYM2k3DsbVpchWf), this is defined by a recursive formula. The general version is not implemented yet.
- `is_channel`: check if the Choi operator is a quantum channel. It is equivalent to `is_sdp and is_causal`.
- `is_unital`: check if the Choi operator is unital. This can be `False` for some channels.

In [6]:
# The tensor structure of the Choi matrix
# For a quantum Channle, the Choi matrix is a 4-tensor:
# In qubit systems, the input and output are assuemed to be the same.
print('Given a quatnum channel defined on 1 qubit:')
print(f'The total dimension is {depolar_choi.dim}')
print(f'The partition of the systems is {depolar_choi.partition}, which corresponds to the shape {depolar_choi.mat.shape}')
print(f'The output system is set as {depolar_choi.sys_out}\n')

print(f'A quantum channel is hermitian: {depolar_choi.is_hermitian}')
print(f'A quantum channel is semidefinite postive: {depolar_choi.is_sdp}\n')

print(f'The depolarizing channel is causal: {depolar_choi.is_causal}')
print(f'The depolarizing channel is unital: {depolar_choi.is_unital}')

Given a quatnum channel defined on 1 qubit:
The total dimension is 4
The partition of the systems is (2, 2), which corresponds to the shape (2, 2, 2, 2)
The output system is set as (False, True)

A quantum channel is hermitian: True
A quantum channel is semidefinite postive: True

The depolarizing channel is causal: True
The depolarizing channel is unital: True


### Methods

The `QuantumNetwork` class has the following methods:
- `__add__`: add two `QuantumNetwork` objects. The result is a `QuantumNetwork` object. The two objects must have the same partition.
- `__matmul__`: link product of two `QuantumNetwork` objects. The result is a `QuantumNetwork` object. The output of the first network should match the input of the second network. Otherwise, use `link` method to specify the indexes.
- `link`: link two `QuantumNetwork` objects. The result is a `QuantumNetwork` object.
- `apply`: apply a `QuantumNetwork` object to a `np.ndarray` object. The result is a `np.ndarray` object.

First, we apply a quantum channel to a quantum state. The result is a quantum state.

In [7]:
state_out = test_ch.apply_density_matrix(backend, state, 1)
# Test `apply`` method
choi_op = depolar_choi.apply(state)
print(f'Applying the quantum channel to a quantu state (`np.ndarray`) gives\n{choi_op}')
assert np.allclose(choi_op, state_out)
# Test `link` method, note that the default data of quantum network is a 4-tensor, so we need to faltten it to compare with the output state
assert np.allclose(state_choi.link(depolar_choi,'ij,jk->ik').mat.flatten(), state_out.flatten())
# Test the ``@`` operator on states
assert np.allclose((state_choi @ depolar_choi).mat.flatten(), state_out.flatten())

print(f'The output of the link product is a quantum state: {state_choi @ depolar_choi}')
print(f'The data of the output state is \n{(state_choi @ depolar_choi).mat}')


Applying the quantum channel to a quantu state (`np.ndarray`) gives
[[0.54454104+0.j         0.1091789 -0.07840543j]
 [0.1091789 +0.07840543j 0.45545896+0.j        ]]
The output of the link product is a quantum state: J[1 -> 2]
The data of the output state is 
[[[[0.54454104+0.j         0.1091789 -0.07840543j]]

  [[0.1091789 +0.07840543j 0.45545896+0.j        ]]]]


The same method can be used for the composition of quantum channels

In [8]:
U = random_unitary(4)
V = random_unitary(4)

ChoiU = QuantumNetwork(U, (4,4), is_pure=True)
ChoiV = QuantumNetwork(V, (4,4), is_pure=True)

# Note that the when represented in unitary, the evolution is from right to left
ChoiUV = QuantumNetwork(V@U, (4,4), is_pure=True)

assert ChoiU.is_hermitian
assert ChoiU.is_causal
assert ChoiU.is_unital
assert ChoiU.is_sdp

# Test `link` method for channels
assert np.allclose(ChoiU.link(ChoiV).mat, ChoiUV.full)
assert np.allclose(ChoiU.link(ChoiV,'ij,jk->ik').mat, ChoiUV.full)

# Test the ``@`` operator
assert np.allclose((ChoiU @ ChoiV).mat, ChoiUV.full)


We can create a **mixed unitary channel** by adding two unitary channels.

In [9]:
mix_ch = (ChoiU + ChoiV)/2
print(mix_ch)

print(f'The mixed unitary channel is a channel: {mix_ch.is_channel}')
print(f'The mixed unitary channel is always unital: {mix_ch.is_unital}')

J[4 -> 4]
The mixed unitary channel is a channel: True
The mixed unitary channel is always unital: True


## Example

In 3-d, a unital channel may not be a mixed unitary channel.

> Example 4.3 in (Watrous, John. The theory of quantum information. Cambridge university press, 2018.)

In [10]:
A1 = np.array([
    [0,0,0],
    [0,0,1/np.sqrt(2)],
    [0,-1/np.sqrt(2),0],
])
A2 = np.array([
    [0,0,1/np.sqrt(2)],
    [0,0,0],
    [-1/np.sqrt(2),0,0],
])
A3 = np.array([
    [0,1/np.sqrt(2),0],
    [-1/np.sqrt(2),0,0],
    [0,0,0],
])

Choi1 = QuantumNetwork(A1, (3,3), is_pure=True)*3
Choi2 = QuantumNetwork(A2, (3,3), is_pure=True)*3
Choi3 = QuantumNetwork(A3, (3,3), is_pure=True)*3

The three channels are pure but not unital. Which means they are not unitary.

In [11]:
print(Choi1.is_unital)
print(Choi2.is_unital)
print(Choi3.is_unital)

False
False
False


However, the mixture of the three operators are unital.
As the matrices are orthogonal, they are the extreme points of the convex set of the unital channels.
Therefore, this mixed channel is not a mixed unitary channel.

In [12]:
Choi = Choi1/3 + Choi2/3 + Choi3/3

print(Choi.is_unital)

True
